In [509]:
import numpy as np
import time

In [510]:
with open("d17.txt", "r") as f:
    wind_txt = f.read()
#wind_txt = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"
len(wind_txt)

10091

In [511]:
def print_array(array):
    print("".join(np.binary_repr(row, 7) + "\n" for row in array))

In [512]:
dtype=np.int8
base_rocks = [np.array([0b0011110,    # - shape
                        0b0000000,
                        0b0000000,
                        0b0000000], dtype=dtype),
              np.array([0b0001000,    # + shape
                        0b0011100,
                        0b0001000,
                        0b0000000], dtype=dtype),
              np.array([0b0011100,    # L shape
                        0b0000100,
                        0b0000100,
                        0b0000000], dtype=dtype),   
              np.array([0b0010000,    # | shape
                        0b0010000,
                        0b0010000,
                        0b0010000], dtype=dtype),
              np.array([0b0011000,    # square shape
                        0b0011000,
                        0b0000000,
                        0b0000000], dtype=dtype)]
right_border =np.array([0b0000001,    
                        0b0000001,
                        0b0000001,
                        0b0000001], dtype=dtype)

rocks = []
for base_rock in base_rocks:
    rock_dict = {}
    rock = np.left_shift(base_rock, 2)
    for shift in range(-2, 10, 1):
        rock_dict[shift] = rock
        if np.any(np.bitwise_and(rock, right_border)):
            break
        rock = np.right_shift(rock, 1)
    else:
        raise ValueError("Didn't hit border!")
    rocks.append(rock_dict)

for rock_dict in rocks:
    print(rock_dict.keys())

dict_keys([-2, -1, 0, 1])
dict_keys([-2, -1, 0, 1, 2])
dict_keys([-2, -1, 0, 1, 2])
dict_keys([-2, -1, 0, 1, 2, 3, 4])
dict_keys([-2, -1, 0, 1, 2, 3])


In [513]:
sta = np.zeros(shape=(10000,), dtype=dtype)
sta[0] = np.array([0b1111111], dtype=dtype)
sta[:5]

array([127,   0,   0,   0,   0], dtype=int8)

In [514]:
height = 0
def update_height(height):
    while sta[height+1]:
        height += 1
    return height
height = update_height(height)
height

0

In [515]:
t0 = time.perf_counter()
N_ROUNDS = 2022
SPACING_HEIGHT = 4      # rocks appear with 3 rows of spacing (air) above the current height
BLOCK_HEIGHT = 4        # height of the comparison block (=4)
N_ROCKS = len(rocks)
N_WIND = len(wind_txt)
i_wind = 0              # wind vector index
for i_round in range(N_ROUNDS):
    rock_dict = rocks[i_round%N_ROCKS]
    i_col = 0           # column index
    dh = SPACING_HEIGHT 
    while True:
        h = height + dh
        # move sideways
        wind_chr = wind_txt[i_wind%N_WIND]
        i_wind += 1
        moved_sideways = False
        if wind_chr == "<": 
            if i_col-1 in rock_dict.keys():
                i_col -= 1
                moved_sideways = True
        else:
            if i_col+1 in rock_dict.keys():
                i_col += 1
                moved_sideways = True
        
        # if rock hits the stack    
        if moved_sideways:
            #print(f"{height=}, {dh=}, {h=}")
            if np.any(np.bitwise_and(sta[h:h+BLOCK_HEIGHT], rock_dict[i_col])):
                if wind_chr == ">": 
                    i_col -= 1
                else:
                    i_col += 1     
                       
        # move down
        rock = rock_dict[i_col]
        if np.any(np.bitwise_and(sta[h-1:h-1+BLOCK_HEIGHT], rock)):
            sta[h:h+BLOCK_HEIGHT] = np.bitwise_or(sta[h:h+BLOCK_HEIGHT], rock)
            height = update_height(height)
            break
        dh -= 1
        
height, time.perf_counter() - t0

(3127, 0.3059145000006538)

In [520]:
1e12/2022*0.3/3600/24/365

4.704716911537773